## Mixed Integer Linear Programming

### Environment Setup

First, let's check if we have a GPU available in system. This is only applicable if you are running server through this notebook. Else comment the next cell.

In [1]:
import subprocess
import html
from IPython.display import display, HTML

def check_gpu():
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, timeout=5)
        result.check_returncode()
        lines = result.stdout.splitlines()
        gpu_info = lines[2] if len(lines) > 2 else "GPU detected"
        gpu_info_escaped = html.escape(gpu_info)
        display(HTML(f"""
        <div style="border:2px solid #4CAF50;padding:10px;border-radius:10px;background:#e8f5e9;">
            <h3>✅ GPU is enabled</h3>
            <pre>{gpu_info_escaped}</pre>
        </div>
        """))
        return True
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired, FileNotFoundError, IndexError) as e:
        display(HTML("""
        <div style="border:2px solid red;padding:15px;border-radius:10px;background:#ffeeee;">
            <h3>⚠️ GPU not detected!</h3>
            <p>This notebook requires a <b>GPU runtime</b>.</p>

            <h4>If running in Google Colab:</h4>
            <ol>
              <li>Click on <b>Runtime → Change runtime type</b></li>
              <li>Set <b>Hardware accelerator</b> to <b>GPU</b></li>
              <li>Then click <b>Save</b> and <b>Runtime → Restart runtime</b>.</li>
            </ol>

            <h4>If running in Docker:</h4>
            <ol>
              <li>Ensure you have <b>NVIDIA Docker runtime</b> installed (<code>nvidia-docker2</code>)</li>
              <li>Run container with GPU support: <code>docker run --gpus all ...</code></li>
              <li>Or use: <code>docker run --runtime=nvidia ...</code> for older Docker versions</li>
              <li>Verify GPU access: <code>docker run --gpus all nvidia/cuda:12.0.0-base-ubuntu22.04 nvidia-smi</code></li>
            </ol>

            <p><b>Additional resources:</b></p>
            <ul>
              <li><a href="https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html" target="_blank">NVIDIA Container Toolkit Installation Guide</a></li>
            </ul>
        </div>
        """))
        return False

check_gpu()

True

#### Install dependencies



In [2]:
# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed
!pip uninstall -y cuda-python cuda-bindings cuda-core
!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-server-cu12 nvidia-nvjitlink-cu12 rapids-logger==0.1.19 cuopt-sh-client
# !pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-server-cu13 nvidia-nvjitlink-cu13 rapids-logger==0.1.19 cuopt-sh-client

Found existing installation: cuda-python 12.9.5
Uninstalling cuda-python-12.9.5:
  Successfully uninstalled cuda-python-12.9.5
Found existing installation: cuda-bindings 12.9.5
Uninstalling cuda-bindings-12.9.5:
  Successfully uninstalled cuda-bindings-12.9.5
Found existing installation: cuda-core 0.3.2
Uninstalling cuda-core-0.3.2:
  Successfully uninstalled cuda-core-0.3.2
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.3/123.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 241.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 252.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 280.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 232.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.9/515.9 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.4/880.4 kB 260.9 MB/s

#### Run cuOpt server in background

Please disable the next cell in case you are trying to test against a server running on a different machine.

In [1]:
import subprocess
import atexit
import signal
import os
import psutil
import requests
import time

host = "localhost"
port = "5000"

def is_cuopt_server_running():
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            cmdline = proc.info['cmdline']
            if 'python3' in proc.info['name'] and cmdline:  # Check if cmdline exists and is not None
                if any('cuopt_server.cuopt_service' in cmd for cmd in cmdline):
                    return True
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return False

def wait_for_server_health(timeout=30):
    """Wait for server to be healthy, return True if healthy, False if timeout"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            response = requests.get(f'http://{host}:{port}/cuopt/health', timeout=5)
            if response.status_code == 200:
                print("Server is healthy!")
                return True
        except requests.exceptions.RequestException:
            pass

        print(f"Waiting for server to be ready... ({int(time.time() - start_time)}s elapsed)")
        time.sleep(1)

    print(f"Server health check timed out after {timeout} seconds")
    return False

# Only start server if it's not already running
if not is_cuopt_server_running():
    # Start cuOpt server in background
    server_process = subprocess.Popen(['python3', '-m', 'cuopt_server.cuopt_service', '-i', host, '-p', port],
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE)

    def cleanup_server():
        try:
            server_process.terminate()
            server_process.wait(timeout=5)  # Wait up to 5 seconds for graceful termination
            print("Terminated cuOpt server")
        except (subprocess.TimeoutExpired, ProcessLookupError):
            # If graceful termination fails, force kill
            try:
                server_process.kill()
                print("Killed cuOpt server")
            except ProcessLookupError:
                pass

    atexit.register(cleanup_server)
    print("Started cuOpt server")
else:
    print("cuOpt server is already running")

if not wait_for_server_health():
        print("Warning: Server may not be fully ready")

Started cuOpt server
Waiting for server to be ready... (0s elapsed)
Waiting for server to be ready... (1s elapsed)
Server is healthy!


In [3]:
# # This is the original question in the example that can be solved

# problem_data = {}

# offsets = [0, 2, 4]
# indices = [0, 1, 0, 1]
# coefficients = [2.0, 4.0, 3.0, 2.0]

# problem_data["csr_constraint_matrix"] = {
#     "offsets" : offsets,
#     "indices" : indices,
#     "values"  :coefficients
# }

# upper_bounds = ["inf", 190.0]
# lower_bounds = [230.0, "ninf"]

# problem_data["constraint_bounds"] = {
#    "upper_bounds" : upper_bounds,
#    "lower_bounds" : lower_bounds
# }

# var_upper_bounds = ["inf", "inf"]
# var_lower_bounds = [0.0, 0.0]

# problem_data["variable_bounds"] = {
#    "upper_bounds" : var_upper_bounds,
#    "lower_bounds" : var_lower_bounds
# }

# objective_coefficients = [5.0, 3.0]
# maximize = True

# problem_data["objective_data"] = {
#     "coefficients" : objective_coefficients,
#     "scalability_factor" : 1.0,
#     "offset" : 0.0
# }

# problem_data["maximize"] = maximize

# problem_data["variable_names"] = ["x", "y"]

# problem_data["variable_types"] = ["I", "C"]

# solver_config  = {
#     "time_limit" : 1.0
# }

# problem_data["solver_config"] = solver_config

# from cuopt_sh_client import CuOptServiceSelfHostClient
# import json
# import time

# # If cuOpt is not running on localhost:5000, edit ip and port parameters
# # polling_timeout set to None will cause the client to poll automatically
# # until there is a result or an error occurs. You may set polling_timeout
# # to a numeric value in seconds (default is 600)
# cuopt_service_client = CuOptServiceSelfHostClient(
#     ip=host,
#     port=port,
#     polling_timeout=None
# )

# solution = cuopt_service_client.get_LP_solve(problem_data, response_type="dict")

# print(json.dumps(solution, indent=4))

{
    "response": {
        "solver_response": {
            "status": "Optimal",
            "solution": {
                "problem_category": "MIP",
                "primal_solution": [
                    37.0,
                    39.5
                ],
                "dual_solution": null,
                "primal_objective": 303.5,
                "dual_objective": null,
                "solver_time": 0.0,
                "solved_by_pdlp": null,
                "vars": {
                    "x": 37.0,
                    "y": 39.5
                },
                "lp_statistics": {},
                "reduced_cost": null,
                "milp_statistics": {
                    "mip_gap": 0.0,
                    "solution_bound": 303.5,
                    "presolve_time": 0.030472612,
                    "max_constraint_violation": 0.0,
                    "max_int_violation": 0.0,
                    "max_variable_bound_violation": 0.0,
                    "num_nodes": 0,
   

### License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [4]:
!pip install --extra-index-url https://pypi.nvidia.com cuopt-mps-parser

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [5]:
import cuopt_mps_parser
from cuopt_sh_client import CuOptServiceSelfHostClient
import json
import time
from cuopt_sh_client import ThinClientSolverSettings

data = "sample.mps"

mps_data = """
NAME           markshare2
ROWS
 N  obj
 E  c1
 E  c2
 E  c3
 E  c4
 E  c5
 E  c6
 E  c7
COLUMNS
    x1        obj                  1   c1                   1
    x2        obj                 -1   c1                   1
    x3        obj                  1   c2                   1
    x4        obj                 -1   c2                   1
    x5        obj                  1   c3                   1
    x6        obj                 -1   c3                   1
    x7        obj                  1   c4                   1
    x8        obj                 -1   c4                   1
    x9        obj                  1   c5                   1
    x10       obj                 -1   c5                   1
    x11       obj                  1   c6                   1
    x12       obj                 -1   c6                   1
    x13       obj                  1   c7                   1
    x14       obj                 -1   c7                   1
    MARK0000  'MARKER'                 'INTORG'
    x15       c1                  74   c2                  20
    x15       c3                  85   c4                  13
    x15       c5                  35   c6                  86
    x15       c7                  41
    x16       c1                  49   c2                   7
    x16       c3                  47   c4                  71
    x16       c5                  61   c6                   8
    x16       c7                  64
    x17       c1                  12   c2                  68
    x17       c3                  67   c4                  78
    x17       c5                  66   c6                  44
    x17       c7                  82
    x18       c1                  93   c2                  69
    x18       c3                  59   c4                  84
    x18       c5                  78   c6                  96
    x18       c7                  24
    x19       c1                  56   c2                  95
    x19       c3                  84   c4                  56
    x19       c5                  46   c6                  64
    x19       c7                  48
    x20       c1                  16   c2                  64
    x20       c3                  59   c4                  66
    x20       c5                  89   c6                  65
    x20       c7                  41
    x21       c1                  39   c2                  76
    x21       c3                  19   c4                   8
    x21       c5                  61   c6                  68
    x21       c7                  29
    x22       c1                  77   c2                  12
    x22       c3                   8   c4                  68
    x22       c5                  25   c6                  53
    x22       c7                  93
    x23       c1                  56   c2                  45
    x23       c3                  50   c4                  48
    x23       c5                  55   c6                  19
    x23       c7                  64
    x24       c1                  73   c2                  43
    x24       c3                  66   c4                  28
    x24       c5                  16   c6                  33
    x24       c7                  39
    x25       c1                   1   c2                  83
    x25       c3                   5   c4                  33
    x25       c5                  81   c6                  28
    x25       c7                  92
    x26       c1                   3   c2                  15
    x26       c3                  51   c4                  34
    x26       c5                  35   c6                  42
    x26       c7                  86
    x27       c1                  68   c2                  90
    x27       c3                  51   c4                   8
    x27       c5                  96   c6                  72
    x27       c7                  64
    x28       c1                  61   c2                  10
    x28       c3                  64   c4                  99
    x28       c5                  23   c6                  39
    x28       c7                  45
    x29       c1                   8   c2                  96
    x29       c3                  64   c4                  80
    x29       c5                  83   c6                   5
    x29       c7                  87
    x30       c1                  55   c2                  98
    x30       c3                  53   c4                  74
    x30       c5                  39   c6                  77
    x30       c7                  34
    x31       c1                  18   c2                  53
    x31       c3                  61   c4                   2
    x31       c5                  14   c6                  37
    x31       c7                  39
    x32       c1                  21   c2                   1
    x32       c3                  45   c4                  10
    x32       c5                  53   c6                  89
    x32       c7                  88
    x33       c1                  57   c2                   2
    x33       c3                   3   c4                  96
    x33       c5                  23   c6                   7
    x33       c7                  99
    x34       c1                  98   c2                  58
    x34       c3                  76   c4                  41
    x34       c5                  23   c6                  78
    x34       c7                  63
    x35       c1                  58   c2                  24
    x35       c3                  17   c4                  98
    x35       c5                  93   c6                  10
    x35       c7                  85
    x36       c1                  57   c2                  90
    x36       c3                  54   c4                  74
    x36       c5                  38   c6                  78
    x36       c7                  48
    x37       c1                  46   c2                  29
    x37       c3                  13   c4                  39
    x37       c5                  15   c6                  10
    x37       c7                  83
    x38       c1                  72   c2                  57
    x38       c3                  89   c4                  91
    x38       c5                  20   c6                  96
    x38       c7                  88
    x39       c1                   6   c2                  19
    x39       c3                  68   c4                  85
    x39       c5                  19   c6                  55
    x39       c7                  85
    x40       c1                  16   c2                  73
    x40       c3                  57   c4                  95
    x40       c5                  28   c6                   1
    x40       c7                   5
    x41       c1                  76   c2                  89
    x41       c3                   4   c4                  96
    x41       c5                  79   c6                  64
    x41       c7                  14
    x42       c1                  21   c2                  31
    x42       c3                  24   c4                   1
    x42       c5                  51   c6                  61
    x42       c7                  31
    x43       c1                  78   c2                  12
    x43       c3                  96   c4                  80
    x43       c5                  24   c6                  63
    x43       c7                  12
    x44       c1                  18   c2                  34
    x44       c3                  81   c4                  90
    x44       c5                   6   c6                  90
    x44       c7                  93
    x45       c1                  11   c2                  67
    x45       c3                  36   c4                  97
    x45       c5                   3   c6                  22
    x45       c7                  55
    x46       c1                  58   c2                  48
    x46       c3                  54   c4                  36
    x46       c5                  47   c6                  78
    x46       c7                   1
    x47       c1                  59   c2                  11
    x47       c3                   3   c4                   7
    x47       c5                  61   c6                  92
    x47       c7                   2
    x48       c1                  25   c2                  22
    x48       c3                  82   c4                  69
    x48       c5                  60   c6                  25
    x48       c7                  22
    x49       c1                  32   c2                  36
    x49       c3                  33   c4                   9
    x49       c5                  71   c6                  24
    x49       c7                  93
    x50       c1                  14   c2                  78
    x50       c3                  88   c4                   9
    x50       c5                  63   c6                  65
    x50       c7                  49
    x51       c1                  16   c2                  75
    x51       c3                   1   c4                  93
    x51       c5                  26   c6                   6
    x51       c7                  35
    x52       c1                   3   c2                  52
    x52       c3                  29   c4                  94
    x52       c5                  66   c6                  68
    x52       c7                  25
    x53       c1                  60   c2                  95
    x53       c3                   4   c4                  44
    x53       c5                  71   c6                  66
    x53       c7                  39
    x54       c1                  12   c2                  57
    x54       c3                  48   c4                  36
    x54       c5                  63   c6                  66
    x54       c7                   1
    x55       c1                   7   c2                  62
    x55       c3                  51   c4                  71
    x55       c5                  56   c6                   1
    x55       c7                  77
    x56       c1                  42   c2                  94
    x56       c3                  14   c4                  37
    x56       c5                  32   c6                  67
    x56       c7                  43
    x57       c1                  98   c2                  10
    x57       c3                  86   c4                  72
    x57       c5                  39   c6                  78
    x57       c7                   7
    x58       c1                  34   c2                  42
    x58       c3                  64   c4                  38
    x58       c5                  31   c6                  21
    x58       c7                  42
    x59       c1                  33   c2                  89
    x59       c3                  73   c4                  74
    x59       c5                  64   c6                  47
    x59       c7                  36
    x60       c1                  16   c2                  11
    x60       c3                  78   c4                  89
    x60       c5                  89   c6                  17
    x60       c7                  63
    x61       c1                  97   c2                  77
    x61       c3                  45   c4                  37
    x61       c5                  62   c6                  89
    x61       c7                   5
    x62       c1                  63   c2                  85
    x62       c3                  65   c4                  24
    x62       c5                  68   c6                  77
    x62       c7                   8
    x63       c1                  66   c2                  30
    x63       c3                  30   c4                  88
    x63       c5                  59   c6                  88
    x63       c7                  43
    x64       c1                  28   c2                  82
    x64       c3                  52   c4                  77
    x64       c5                  71   c6                  54
    x64       c7                  18
    x65       c1                  57   c2                  20
    x65       c3                   6   c4                  61
    x65       c5                  48   c6                  10
    x65       c7                  60
    x66       c1                  19   c2                  52
    x66       c3                  78   c4                  80
    x66       c5                  76   c6                  87
    x66       c7                  47
    x67       c1                  74   c2                  78
    x67       c3                   9   c4                   2
    x67       c5                  96   c6                  88
    x67       c7                  47
    x68       c1                  44   c2                   6
    x68       c3                  19   c4                  60
    x68       c5                  74   c6                  80
    x68       c7                  46
    x69       c1                  45   c2                  57
    x69       c3                  87   c4                  87
    x69       c5                  61   c6                  76
    x69       c7                  45
    x70       c1                  49   c2                  65
    x70       c3                  73   c4                  80
    x70       c5                  21   c6                   9
    x70       c7                  38
    x71       c1                  76   c2                  79
    x71       c3                  10   c4                  74
    x71       c5                  46   c6                  83
    x71       c7                   9
    x72       c1                  74   c2                  83
    x72       c3                  87   c4                  42
    x72       c5                  18   c6                  95
    x72       c7                  37
    x73       c1                   9   c2                  16
    x73       c3                  33   c4                   2
    x73       c5                  23   c6                  86
    x73       c7                   8
    x74       c1                  44   c2                  67
    x74       c3                   1   c4                  37
    x74       c5                  24   c6                  24
    x74       c7                  82
    MARK0001  'MARKER'                 'INTEND'
RHS
    rhs       c1                1324   c2                1554
    rhs       c3                1429   c4                1686
    rhs       c5                1482   c6                1613
    rhs       c7                1424
BOUNDS
 FX bnd       x2                   0
 FX bnd       x4                   0
 FX bnd       x6                   0
 FX bnd       x8                   0
 FX bnd       x10                  0
 FX bnd       x12                  0
 FX bnd       x14                  0
 UP bnd       x15                  1
 UP bnd       x16                  1
 UP bnd       x17                  1
 UP bnd       x18                  1
 UP bnd       x19                  1
 UP bnd       x20                  1
 UP bnd       x21                  1
 UP bnd       x22                  1
 UP bnd       x23                  1
 UP bnd       x24                  1
 UP bnd       x25                  1
 UP bnd       x26                  1
 UP bnd       x27                  1
 UP bnd       x28                  1
 UP bnd       x29                  1
 UP bnd       x30                  1
 UP bnd       x31                  1
 UP bnd       x32                  1
 UP bnd       x33                  1
 UP bnd       x34                  1
 UP bnd       x35                  1
 UP bnd       x36                  1
 UP bnd       x37                  1
 UP bnd       x38                  1
 UP bnd       x39                  1
 UP bnd       x40                  1
 UP bnd       x41                  1
 UP bnd       x42                  1
 UP bnd       x43                  1
 UP bnd       x44                  1
 UP bnd       x45                  1
 UP bnd       x46                  1
 UP bnd       x47                  1
 UP bnd       x48                  1
 UP bnd       x49                  1
 UP bnd       x50                  1
 UP bnd       x51                  1
 UP bnd       x52                  1
 UP bnd       x53                  1
 UP bnd       x54                  1
 UP bnd       x55                  1
 UP bnd       x56                  1
 UP bnd       x57                  1
 UP bnd       x58                  1
 UP bnd       x59                  1
 UP bnd       x60                  1
 UP bnd       x61                  1
 UP bnd       x62                  1
 UP bnd       x63                  1
 UP bnd       x64                  1
 UP bnd       x65                  1
 UP bnd       x66                  1
 UP bnd       x67                  1
 UP bnd       x68                  1
 UP bnd       x69                  1
 UP bnd       x70                  1
 UP bnd       x71                  1
 UP bnd       x72                  1
 UP bnd       x73                  1
 UP bnd       x74                  1
ENDATA
"""

with open(data, "w") as file:
    file.write(mps_data)

# Parse the MPS file and measure the time spent
parse_start = time.time()
data_model = cuopt_mps_parser.ParseMps(data)
parse_time = time.time() - parse_start

problem_data = cuopt_mps_parser.toDict(data_model)
print(problem_data)
print(f"Parsing time: {parse_time} seconds")





{'csr_constraint_matrix': {'offsets': array([  0,  62, 124, 186, 248, 310, 372, 434], dtype=int32), 'indices': array([ 0,  1, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
       46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,  2,  3, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73,  4,  5, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,  6,
        7, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2

In [ ]:
# If cuOpt is not running on localhost:5000, edit ip and port parameters
# polling_timeout set to None will cause the client to poll automatically
# until there is a result or an error occurs. You may set polling_timeout
# to a numeric value in seconds (default is 600)
cuopt_service_client = CuOptServiceSelfHostClient(
    ip=host,
    port=port,
    timeout_exception=False
)

solution = cuopt_service_client.get_LP_solve(data_model, response_type="dict")

print(json.dumps(solution, indent=4))


In [2]:
from cuopt_sh_client import CuOptServiceSelfHostClient
import json
import time


data = "sample.mps"

mps_data = """* optimize
*  cost = 0.2 * VAR1 + 0.1 * VAR2
* subject to
*  3 * VAR1 + 4 * VAR2 <= 5.4
*  2.7 * VAR1 + 10.1 * VAR2 <= 4.9
NAME   good-1
ROWS
 N  COST
 L  ROW1
 L  ROW2
COLUMNS
    VAR1      COST      0.2
    VAR1      ROW1      3              ROW2      2.7
    VAR2      COST      0.1
    VAR2      ROW1      4              ROW2      10.1
RHS
    RHS1      ROW1      5.4            ROW2      4.9
ENDATA
"""

with open(data, "w") as file:
    file.write(mps_data)

# If cuOpt is not running on localhost:5000, edit `ip` and `port` parameters
cuopt_service_client = CuOptServiceSelfHostClient(
    ip="localhost",
    port=5000,
    timeout_exception=False
)

solution = cuopt_service_client.get_LP_solve(data, response_type="dict")

print(json.dumps(solution, indent=4))

{
    "response": {
        "solver_response": {
            "status": "Optimal",
            "solution": {
                "problem_category": "LP",
                "primal_solution": [
                    0.0,
                    0.0
                ],
                "dual_solution": [
                    0.0,
                    0.0
                ],
                "primal_objective": 0.0,
                "dual_objective": 0.0,
                "solver_time": 0.078884972,
                "solved_by_pdlp": false,
                "vars": {
                    "VAR1": 0.0,
                    "VAR2": 0.0
                },
                "lp_statistics": {
                    "primal_residual": 0.0,
                    "dual_residual": 0.0,
                    "gap": 0.0,
                    "nb_iterations": 0
                },
                "reduced_cost": [
                    0.2,
                    0.1
                ],
                "milp_statistics": {}
            }
  

In [6]:
# Direct loading from simple MPS file
from cuopt_sh_client import CuOptServiceSelfHostClient
import json
import time

data = "simpleExample.mps"

# If cuOpt is not running on localhost:5000, edit `ip` and `port` parameters
cuopt_service_client = CuOptServiceSelfHostClient(
    ip="localhost",
    port=5000,
    timeout_exception=False
)


solution = cuopt_service_client.get_LP_solve(data, response_type="dict")

print(json.dumps(solution, indent=4))

{
    "response": {
        "solver_response": {
            "status": "Optimal",
            "solution": {
                "problem_category": "LP",
                "primal_solution": [
                    0.0,
                    0.0
                ],
                "dual_solution": [
                    0.0,
                    0.0
                ],
                "primal_objective": 0.0,
                "dual_objective": 0.0,
                "solver_time": 0.008182336,
                "solved_by_pdlp": false,
                "vars": {
                    "VAR1": 0.0,
                    "VAR2": 0.0
                },
                "lp_statistics": {
                    "primal_residual": 0.0,
                    "dual_residual": 0.0,
                    "gap": 0.0,
                    "nb_iterations": 0
                },
                "reduced_cost": [
                    0.2,
                    0.1
                ],
                "milp_statistics": {}
            }
  